# **Journal Results**

- **Author:** Nedal M. Benelmekki
- **Date:** 21/10/2025
- **Description:** New Journal results. 

## **External Imports**

### **Add Project Root**

In [ ]:
# -*- coding: utf-8 -*-
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) 

### **Import Necessary Modules**

In [ ]:
from style import style as st
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import os

## **User Configuration**

In [ ]:
# Results directory
BASE_DIR = Path("/home/nedal/Desktop/RFID/ICASSP2026/Bayesian-Enhanced-AoA-Estimator/results/final-trained/final-results")

# Output directory
OUTPUT_DIR = Path("/home/nedal/Desktop/RFID/ICASSP2026/Bayesian-Enhanced-AoA-Estimator/final-results")
OUTPUT_DIR.mkdir(exist_ok=True)

# Regex patterns in .txt files
pattern_main = re.compile(r"MAE:\s*([\d\.]+)\u00B0\s*RMSE:\s*([\d\.]+)\u00B0")
pattern_phys = re.compile(r"(\w+): MAE=([\d\.]+)\u00B0, RMSE=([\d\.]+)\u00B0")

## **Data Collection**

### **Sweep Files & Collect Data**

In [ ]:
records = []

# Mapping dictionaries for the antenna-array techniques
PRIOR_MAP = {
    'phase': 'PD',
    'weighted': 'WDS',
    'ds': 'DS',
    'music': 'MUSIC',
}
METHOD_MAP = {'svi': 'SVI', 'mcmc': 'MCMC'}

def map_feature_mode(s: str) -> str:
    s_l = s.lower().replace('-', '_').replace(' ', '')
    if s_l in ('full', 'full_model', 'fullmodel'):
        return 'Full Model'
    if s_l in ('width', 'width_model', 'widthmodel'):
        return 'Width Model'
    if s_l in ('sensor', 'sensor_model', 'sensormodel'):
        return 'Sensor Model'
    # "Error" case
    return ' '.join(part.capitalize() for part in s.replace('_', ' ').split())

for d in BASE_DIR.iterdir():
    if not d.is_dir():
        continue  
    try:
        # Parse name
        dir_name = d.name
        
        # Extract replica
        if not dir_name.endswith(('r1', 'r2', 'r3')):
            print(f"[!] Invalid replica format in {dir_name}")
            continue
        # Obtain replica and trim
        r_str    = dir_name[-2:]
        dir_name = dir_name[:-3]
        
        # Extract percentage value
        p_parts = dir_name.split('_p')
        if len(p_parts) != 2:
            print(f"[!] Invalid p-value format in {dir_name}")
            continue
        dir_name = p_parts[0]
        p_str    = 'p' + p_parts[1]
        
        # Extract method (svi or mcmc)
        methods = ['svi', 'mcmc']
        method  = None
        for m in methods:
            if dir_name.endswith(f"_{m}"):
                method   = m
                dir_name = dir_name[:-len(m)-1]
                break  
        if method is None:
            print(f"[!] No valid method found in {d.name}")
            continue
        
        # Extract prior observation
        priors = ['ds', 'music', 'phase', 'weighted']
        prior = None
        for p in priors:
            if dir_name.startswith(f"{p}_"):
                prior = p
                dir_name = dir_name[len(p)+1:]
                break
        if prior is None:
            print(f"[!] No valid prior found in {d.name}")
            continue
        feature_mode_raw = dir_name
        feature_mode_label = map_feature_mode(feature_mode_raw)
        
        # Validate format
        try:
            p_val = int(p_str.replace("p", ""))
            replica = int(r_str.replace("r", ""))
        except ValueError:
            print(f"[!] Invalid p-value or replica in {d.name}")
            continue
            
        # Parse model_summary.txt
        summary_file = d / "model_summary.txt"
        if not summary_file.exists():
            print(f"[!] Missing summary in {d}")
            continue

        content = summary_file.read_text()

        # Extract main performance
        match_main = pattern_main.search(content)
        if not match_main:
            print(f"[!] No main MAE/RMSE in {d}")
            continue

        mae = float(match_main.group(1))
        rmse = float(match_main.group(2))

        # Extract physics-based comparison
        phys_results = dict()
        for match in pattern_phys.findall(content):
            model, mae_val, rmse_val = match
            phys_results[f"{model.lower()}_mae"] = float(mae_val)
            phys_results[f"{model.lower()}_rmse"] = float(rmse_val)

        # Add record with desired labels
        records.append({
            "prior": PRIOR_MAP.get(prior, prior.upper()),
            "features": feature_mode_label,
            "method": METHOD_MAP.get(method, method.upper()),
            "samples_%": p_val,
            "replica": replica,
            "mae": mae,
            "rmse": rmse,
            **phys_results
        })

    except Exception as e:
        print(f"[x] Error parsing {d}: {e}")


### **Build Dataframe**

In [ ]:
df = pd.DataFrame(records)
print(f"[i] Loaded {len(df)} entries")
print(df.head())

## **Preprocesing**

### **Average Across Replicas**

In [ ]:
group_cols = ["prior", "features", "method", "samples_%"]
df_avg     = df.groupby(group_cols).agg({"mae": "mean", "rmse": "mean"}).reset_index()
print("\n[i] Averaged across replicas:")
print(df_avg.head())

## **Results**

### **Table**

In [ ]:
# Create table for all configurations
print("\n[i] Table for publication (sorted by MAE):")
styled_table = df_avg.sort_values("mae").round(4)
print(styled_table.to_string(index=False))

# Export to CSV for easy import into LaTeX or other tools
output_csv = OUTPUT_DIR / "results_table.csv"
styled_table.to_csv(output_csv, index=False)
print(f"\n[i] Exported table to {output_csv}")

### **Training Data vs. Performance**

In [ ]:
plt.figure(figsize=(12,8))

# Plot lines for each configuration
for (prior, method, features), subset in df_avg.groupby(["prior", "method", "features"]):
    subset = subset.sort_values("samples_%")
    plt.plot(
        subset["samples_%"], 
        subset["mae"], 
        label=f"{prior}-{method}-{features}"
    )

plt.xlabel("Training Data (\%)")
plt.ylabel("Mean Absolute Error ($^\circ$)")
plt.title("Performance vs. Training Data Size")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
output_fig = OUTPUT_DIR / "performance_vs_samples.png"
plt.savefig(output_fig, dpi=600)
plt.show()

### **Best Configuration Analysis**

#### **Compute Best Performance**

In [ ]:
# Find and save the best configuration for each sample size
best_by_sample = df_avg.loc[df_avg.groupby("samples_%")["mae"].idxmin()]
print("\n[i] Best configuration for each sample size:")
print(best_by_sample[["samples_%", "prior", "features", "method", "mae", "rmse"]])
best_sample_output = OUTPUT_DIR / "best_by_sample_size.csv"
best_by_sample.to_csv(best_sample_output, index=False)
print(f"[i] Saved best configurations by sample size to {best_sample_output}")

# Find overall best configuration
best_idx = df_avg["mae"].idxmin()
best_config = df_avg.loc[best_idx].to_dict()
print(f"\n[i] Best overall configuration: {best_config['prior']}-{best_config['features']}-{best_config['method']} with {best_config['samples_%']}% samples")

# Get the full data for the best configuration
best_data = df[(df["prior"] == best_config["prior"]) & 
              (df["features"] == best_config["features"]) & 
              (df["method"] == best_config["method"]) & 
              (df["samples_%"] == best_config["samples_%"])]

# Compare with baseline methods
if len([col for col in best_data.columns if col.endswith('_mae')]) > 0:
    baselines = ["music_mae", "ds_mae", "weighted_mae", "phase_mae"]
    baseline_names = ["MUSIC", "DS", "Weighted DS", "Phase"]
    
    improvements = []
    for baseline in baselines:
        if baseline in best_data.columns:
            avg_baseline = best_data[baseline].mean()
            improvement = (avg_baseline - best_data["mae"].mean()) / avg_baseline * 100
            improvements.append((baseline_names[baselines.index(baseline)], avg_baseline, best_data["mae"].mean(), improvement))
    
    print("\n[i] Improvement over baselines:")
    for name, baseline, bayesian, improvement in improvements:
        print(f"{name}: {baseline:.4f} deg ? {bayesian:.4f} deg ({improvement:.2f}% improvement)")
        
    # Save improvement stats
    improvement_output = OUTPUT_DIR / "improvement_summary.txt"
    with open(improvement_output, "w") as f:
        f.write(f"Best configuration: {best_config['prior']}-{best_config['features']}-{best_config['method']} with {best_config['samples_%']}% samples\n")
        f.write(f"MAE: {best_config['mae']:.4f} deg, RMSE: {best_config['rmse']:.4f} deg\n\n")
        f.write("Improvement over baselines:\n")
        for name, baseline, bayesian, improvement in improvements:
            f.write(f"{name}: {baseline:.4f} deg ? {bayesian:.4f} deg ({improvement:.2f}% improvement)\n")
    print(f"[i] Saved improvement summary to {improvement_output}")

#### **Visualization**

In [ ]:
if len([col for col in best_data.columns if col.endswith('_mae')]) > 0:
    plt.figure(figsize=(10, 6))
    
    # Prepare data for plotting
    methods = ["Bayesian"] + [name for name, _, _, _ in improvements]
    errors = [best_data["mae"].mean()] + [baseline for _, baseline, _, _ in improvements]
    
    # Create bar chart
    bars = plt.bar(methods, errors)
    bars[0].set_color('green')
    
    plt.ylabel("Mean Absolute Error ($^\circ$)")
    plt.title(f"Performance Comparison: {best_config['prior']}-{best_config['features']}-{best_config['method']} vs. Baselines")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add error values on top of bars
    for i, v in enumerate(errors):
        plt.text(i, v + 0.1, f"{v:.2f} $^\circ$", ha='center')
    
    # Save figure
    comparison_fig = OUTPUT_DIR / "baseline_comparison.png"
    plt.savefig(comparison_fig, dpi=600, bbox_inches='tight')
    plt.show()
    print(f"[i] Saved baseline comparison chart to {comparison_fig}")


### **Top Perormance Analysis**

In [ ]:
# Filter to avoid geometric model
print(df_avg["features"].unique())
df_width = df_avg[df_avg["features"] == "Width Only"].copy()
if df_width.empty:
    raise ValueError('No rows with features == "Width Only". Check your mapping upstream.')

# Top configurations
top_configs = (
    df_width.groupby(["prior", "method"])["mae"]
    .mean()
    .nsmallest(1)
    .reset_index()
)

top_configs_list = list(zip(top_configs["prior"], top_configs["method"]))

print("\n[i] Top performing configurations:")
for i, (prior, method) in enumerate(top_configs_list, 1):
    print(f"{i}. {prior} - Width Only - {method}")

plt.figure(figsize=(10, 6))

colors  = ["red", "#E69F00", "#009E73", "#D55E00", "#CC79A7", "#56B4E9"]
markers = ["o", "s", "D", "^", "v", "P", "X", "*"]

for i, (prior, method) in enumerate(top_configs_list):
    subset = df_width[(df_width["prior"] == prior) & 
                  (df_width["method"] == method)]
    subset = subset.sort_values("samples_%")
    plt.plot(subset["samples_%"], subset["mae"], linewidth = 4.0, label=f"{prior} - Width Only - {method}", color=colors[i % len(colors)],
        marker=markers[i % len(markers)], markersize=7,)

plt.xlabel("Training Data (\%)", fontsize=18, labelpad=20)
plt.ylabel("Mean Absolute Error ($^\circ$)", fontsize=18, labelpad=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Performance vs. Training Data Size", fontsize=20)
plt.legend(loc='best', fontsize = 16)
plt.grid(True)
plt.tight_layout()
top_perf_fig = OUTPUT_DIR / "top_performers_vs_samples.png"
plt.savefig(top_perf_fig, dpi=600)
plt.show()
print(f"[i] Saved top performers chart to {top_perf_fig}")

### **Prior Type Analysis**

In [ ]:
prior_performance = df_avg.groupby("prior")["mae"].mean().reset_index()
prior_performance = prior_performance.sort_values("mae")

plt.figure(figsize=(8, 5))
bars = plt.bar(prior_performance["prior"], prior_performance["mae"])
plt.ylabel("Mean Absolute Error ($^\circ$)")
plt.title("Performance by Prior Type")
plt.grid(axis='y', linestyle='--', alpha=0.7)

for i, v in enumerate(prior_performance["mae"]):
    plt.text(i, v + 0.02, f"{v:.2f} deg", ha='center')

prior_fig = OUTPUT_DIR / "prior_performance.png"
plt.savefig(prior_fig, dpi=600)
plt.show()
print(f"[i] Saved prior performance chart to {prior_fig}")

### **Sample Efficiency Analysis**

In [ ]:
best_overall_config = best_config["prior"], best_config["method"], best_config["features"]

full_perf = df_avg[(df_avg["prior"] == best_config["prior"]) & 
                 (df_avg["method"] == best_config["method"]) & 
                 (df_avg["features"] == best_config["features"]) &
                 (df_avg["samples_%"] == 100)]["mae"].values

if len(full_perf) > 0:
    full_perf = full_perf[0]
    
    sample_perf = df_avg[(df_avg["prior"] == best_config["prior"]) & 
                        (df_avg["method"] == best_config["method"]) & 
                        (df_avg["features"] == best_config["features"])]
    
    sample_perf = sample_perf.sort_values("samples_%")
    sample_perf["relative_perf"] = full_perf / sample_perf["mae"] * 100
    
    plt.figure(figsize=(8, 5))
    plt.plot(sample_perf["samples_%"], sample_perf["relative_perf"], 
             marker='o', linewidth=2)
    plt.axhline(y=100, color='gray', linestyle='--')
    
    plt.xlabel("Training Data (\%)")
    plt.ylabel("Performance Relative to Full Dataset (\%)")
    plt.title(f"Sample Efficiency: {best_config['prior']}-{best_config['features']}-{best_config['method']}")
    plt.grid(True)
    
    efficiency_fig = OUTPUT_DIR / "sample_efficiency.png"
    plt.savefig(efficiency_fig, dpi=600)
    plt.show()
    print(f"[i] Saved sample efficiency chart to {efficiency_fig}")

### **Method Comparison**

In [ ]:
# Compare SVI vs MCMC performance
method_comparison = df_avg.groupby(["method", "features"])["mae"].mean().reset_index()
method_pivot = method_comparison.pivot(index="features", columns="method", values="mae")

plt.figure(figsize=(10, 6))
ax = method_pivot.plot(kind="bar", figsize=(10, 6), color=["#0A2342", "#17BECF"])
ax.set_ylabel("MAE ($^\circ$)", labelpad=20)
ax.set_xlabel("Feature Set", labelpad=20)
ax.set_title("SVI vs MCMC Performance by Feature Set")
ax.grid(axis='y', linestyle='--', alpha=0.7)

ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
ax.legend(title=None)

plt.subplots_adjust(bottom=0.5)

method_fig = OUTPUT_DIR / "svi_vs_mcmc.png"
plt.savefig(method_fig, dpi=600)
plt.show()
print(f"[i] Saved method comparison chart to {method_fig}")

### **Relative Improvement Heatmap**

In [ ]:
baseline_ref = "phase_mae"
df_improv = df.copy()
df_improv["improvement_%"] = (
    (df_improv[baseline_ref] - df_improv["mae"]) / df_improv[baseline_ref] * 100
)

avg_improv = df_improv.groupby(["prior", "method", "features"])["improvement_%"].mean().reset_index()

plt.figure(figsize=(9,5))
pivot_improv = avg_improv.pivot_table(
    values="improvement_%",
    index="prior",
    columns=["method", "features"]
)
sns.heatmap(pivot_improv, annot=True, fmt=".1f", cmap="YlGnBu", cbar_kws={'label': '% Improvement'})
plt.title(f"Average Improvement over {baseline_ref.split('_')[0].upper()} Baseline")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "improvement_heatmap.png", dpi=600)

### **Prior vs. Posterior**

In [ ]:
# Labels for features
FEATURE_LABELS = {
    "phase1_mean":        r"Phase 1 ($\varphi_1$)",
    "phase2_mean":        r"Phase 2 ($\varphi_2$)",
    "phase_diff":         r"Phase Diff. ($\Delta \varphi$)",
    "mag1_mean":          r"Mag. 1 ($mag_1$)",
    "mag2_mean":          r"Mag. 2 ($mag_2$)",
    "rssi1_mean":         r"RSSI$_1$",
    "rssi2_mean":         r"RSSI$_2$",
    "rssi_diff":          r"$\Delta$ RSSI ",
    "phasor1_real_mean":  r"$\Re\{z_1\}$",
    "phasor1_imag_mean":  r"$\Im\{z_1\}$",
    "phasor2_real_mean":  r"$\Re\{z_2\}$",
    "phasor2_imag_mean":  r"$\Im\{z_2\}$",
    "wavelength":         r"Wavelength ($\lambda$)",
    "distance":           r"Distance ($D$)",
    "width":              r"Width ($W$)",
    "bias":               r"Bias"
}

def _label_for(name: str) -> str:
    return FEATURE_LABELS.get(name, name.replace('_', ' ').title())

def _normal_pdf(x, mu, sd):
    sd = max(float(sd), 1e-12)
    return (1.0 / (sd * np.sqrt(2.0 * np.pi))) * np.exp(-0.5 * ((x - mu) / sd) ** 2)

# Parse summary file
_SUMMARY_RE = re.compile(r"^\s*([A-Za-z0-9_]+)\s*:\s*([+-]?\d+(?:\.\d+)?)\s*±\s*([+-]?\d+(?:\.\d+)?)\s*$")

def load_posteriors_from_summary(summary_path: str):
    """
    Returns a dict: name -> (mean, std) where std is 1σ (converted from '± 2σ')
    """
    out = {}
    with open(summary_path, 'r') as f:
        for line in f:
            m = _SUMMARY_RE.match(line)
            if m:
                name = m.group(1)
                mean = float(m.group(2))
                two_sigma = float(m.group(3))
                std = two_sigma / 2.0
                out[name] = (mean, std)
    return out

# Plot prior vs posterior for selected params (weights & the bias)
def plot_param_priors_vs_posteriors(
    summary_path: str,
    out_path: str,
    w_prior_sd: float = 1.0,
    b_prior_sd: float = 5.0,
    params: list[str] | None = None,
    show_true_lines: bool = False,
    true_vals: dict[str, float] | None = None,
    tight_margin: float = 4.0
):
    """
    Reads posterior means/stds from your summary and overlays the Normal priors.

    - summary_path: path to model_summary.txt
    - out_path:     where to save the figure (png, pdf, etc.)
    - w_prior_sd:   σ_w from your HBLRConfig (for all weights)
    - b_prior_sd:   σ_b from your HBLRConfig (for bias)
    - params:       optional subset of parameter names to plot (order preserved)
    - show_true_lines/true_vals: optional vertical black lines if you have known truths
    - tight_margin: x-range ~ +/- tight_margin * max(prior_sd, post_sd) around means
    """
    posts = load_posteriors_from_summary(summary_path)
    if not posts:
        raise ValueError(f"No parameters parsed from {summary_path}")

    keys = list(posts.keys())
    if params:
        keys = [k for k in params if k in posts]
    else:
        if "Bias" in keys:
            keys = [k for k in keys if k != "Bias"] + ["Bias"]

    n = len(keys)
    ncols = min(4, max(1, int(np.ceil(np.sqrt(n)))))
    nrows = int(np.ceil(n / ncols))

    fig, axes = plt.subplots(nrows, ncols, figsize=(4*ncols + 2, 3*nrows + 1))
    axes = np.atleast_1d(axes).ravel()

    for i, name in enumerate(keys):
        ax = axes[i]
        mean, post_sd = posts[name]
        prior_sd = b_prior_sd if name.lower() == "bias" else w_prior_sd
        prior_mu = 0.0

        span = tight_margin * max(prior_sd, post_sd)
        center = 0.5 * (prior_mu + mean)
        x_min = min(prior_mu, mean) - span
        x_max = max(prior_mu, mean) + span
        if x_max - x_min < 1e-6:
            x_min, x_max = center - span, center + span

        x = np.linspace(x_min, x_max, 800)

        # Prior: red dashed + fill
        prior_pdf = _normal_pdf(x, prior_mu, prior_sd)
        ax.plot(x, prior_pdf, 'r--', lw=2, label='Prior')
        ax.fill_between(x, 0, prior_pdf, color='red', alpha=0.2)

        # Posterior: blue solid + fill
        post_pdf = _normal_pdf(x, mean, post_sd)
        ax.plot(x, post_pdf, 'b-', lw=2, label='Posterior')
        ax.fill_between(x, 0, post_pdf, color='blue', alpha=0.2)

        # “True” vertical line
        if show_true_lines and true_vals and name in true_vals:
            ax.axvline(true_vals[name], color='k', lw=2, label='True')

        ax.set_title(_label_for(name))
        ax.set_xlabel('Weight Value')
        ax.set_ylabel('Density')
        ax.grid(True, alpha=0.3)

        if i == 0:
            ax.legend(loc="upper left")

    # Hide any unused axes
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    fig.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    plt.show()


summary_path = "/home/nedal/Desktop/RFID/ICASSP2026/Bayesian-Enhanced-AoA-Estimator/results/final-trained/final-results/music_full_mcmc_p5_r1/weights_bias_summary.txt"  # adjust path
out_path     = "/home/nedal/Desktop/RFID/ICASSP2026/Bayesian-Enhanced-AoA-Estimator/final-results/prior_vs_posterior_weights.png"

# Option A: plot all parsed parameters (weights + bias)
plot_param_priors_vs_posteriors(
    summary_path=summary_path,
    out_path=out_path,
    w_prior_sd=1.0,
    b_prior_sd=5.0
)